In [9]:
import pandas as pd
import numpy as np
import json
import re

In [21]:
df = pd.read_excel("quora_questions_for_entity.xlsx",sheet_name="Sheet1")
entity_data = {}

In [22]:
ANATOMY = df["ANATOMY"].values.tolist()
ANATOMY.sort(key = lambda s: len(str(s)))
ANATOMY = list(reversed(ANATOMY))
entity_data.update({"ANATOMY":ANATOMY})


DEMOGRAPHIC = df["DEMOGRAPHIC"].values.tolist()
DEMOGRAPHIC.sort(key = lambda s: len(str(s)))
DEMOGRAPHIC = list(reversed(DEMOGRAPHIC))
entity_data.update({"DEMOGRAPHIC":DEMOGRAPHIC})

DEVICES = df["DEVICES"].values.tolist()
DEVICES.sort(key = lambda s: len(str(s)))
DEVICES = list(reversed(DEVICES))
entity_data.update({"DEVICES":DEVICES})

DRUGS = df["DRUGS"].values.tolist()
DRUGS.sort(key = lambda s: len(str(s)))
DRUGS = list(reversed(DRUGS))
entity_data.update({"DRUGS":DRUGS})

FINDINGS = df["FINDINGS"].values.tolist() 
FINDINGS.sort(key = lambda s: len(str(s)))
FINDINGS = list(reversed(FINDINGS))
entity_data.update({"FINDINGS":FINDINGS})

PROBLEMS = df["PROBLEMS"].values.tolist()
PROBLEMS.sort(key = lambda s: len(str(s)))
PROBLEMS = list(reversed(PROBLEMS))
entity_data.update({"PROBLEMS":PROBLEMS})

PROCEDURES = df["PROCEDURES"].values.tolist()
PROCEDURES.sort(key = lambda s: len(str(s)))
PROCEDURES = list(reversed(PROCEDURES))
entity_data.update({"PROCEDURES":PROCEDURES})
#"led" "co" "mct" "aim" "thc" "hla" "atp" "msm" "uk"

In [23]:
stop_words = set(["best", "way", "much", "many", "same", "time", "easy", "ways", "i", "m", "i'", "isn't", "should", "could", "would", "shall", "can", "will", "",
                  "shouldn't", "couldn't", "wouldn't", "shalln't", "can't", "will", "won't", "not", "first", "last", "what", "where", "how", "who", "there", "this",
                  "that", "it", "they", "those", "these", "them", "was", "is", "am", "are", "do", "done", "did","my","name","is","his","her","afraid","ideas","ideal","ill",
                  "effect","iit","eat","induced","day","home","fasting","years","days","hours","year","day","hour","month","months","here","there","it","here","there","much",
                  "many","few","i","me","we","he","she","it","affect","other",
                  "associated","point","egg","his","her","he","she","wake up","air","hands feet","skip",
                  "go","went","gone","pick","treat","treatment"])

In [24]:
df2 = pd.read_excel("diab.xlsx",sheet_name="Sheet1")

In [72]:
questions = df2.loc[:,0].values.tolist()
questions = list(map(lambda x: str(x).lower(),questions))
questions = list(map(lambda x: re.sub(r"\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\}|\||\:|\"|\<|\>|\?|\[|\]|\\|\;|\'|\,|\/|\."," ",str(x)),questions))
questions = list(map(lambda x:re.sub(r"\s{2,}"," ",x),questions))
questions[0:10]

['3 out of 4 of my relatives have died from diabetic complications gangrene amputations or heart attacks before the age of 60 i am a healthy 30 year old how do i assure this does not happen to me ',
 'a lot of discussions are going on on hba1c and diabetes how do i decide about the correct reason to put you in diabetic or non-diabetic ',
 'a patient was diagnosed with left capsuloganglionic hemorrhage what might be the prognosis his age is 60 years and is with associated comorbidities hypertension and diabetes what would be the better plan of treatment ',
 'about 5 months ago i was diagnosed as a type 1 diabetic for some reason the diagnosis has made me paranoid about other aspects of my health and i ll find myself getting incredibly anxious at ridiculous things how can i stop worrying ',
 'according to the most updated knowledge that we possess in 2018 what causes diabetes high blood sugar provided that it does not run in the family ',
 'after 3-4 months can there possibly be any prob

In [68]:
def f(entity_list,sentence,item,entity_name):
    a = re.split(r'\s'+item+r'\s',sentence)
    if len(a) < 3:
        start_index = len(a[0])
        end_index = start_index + len(item)
        entity_list.append((start_index,end_index,entity_name,item))
    else:
#         print("-------------begin----------------------")
        for index,value in enumerate(a[0:len(a)-1]):
            start_index = len(a[index])
            temp = index
            while temp-1 >=0 :
                temp = temp - 1
                start_index =  start_index + len(a[temp])+len(" "+item+" ")
            end_index = start_index + len(item)
            
            entity_list.append((start_index,end_index,entity_name,item))
    return entity_list

In [69]:
# order in which entities are to be tagged
preference_order = ["PROBLEMS","PROCEDURES","DRUGS","DEVICES","FINDINGS","ANATOMY","DEVICES"]

In [70]:
#('Is it recommended to make lumpsum investments in elss mf funds or do a SIP?', {'entities': [(26, 33, 'modeOfInvestment'), (71, 74, 'modeOfInvestment'), (49, 56, 'elss')]})

In [71]:
train_data = []
count = 0
for question in questions:
    question = " "+str(question)+" "
    entity_list = []
#     count += 1
    for entity_ in preference_order:
        for item in entity_data[entity_]:
            if type(item) is float or type(item) is int:
                pass
            else:
                if " "+item+" " in question:
                    if re.search(r"\w+",item):
                        if item not in stop_words:
                            entity_list = f(entity_list,question,item,entity_)
    train_data.append((question.strip(),{"entities":entity_list}))
#     print((question.strip(),{"entities":entity_list}))
#     if count > 10:
#         break
print(len(questions))
print(len(train_data))

2540
2540


In [ ]:
('are 44 units of lantus in the morning and 48 units of lantus at night plus novolog for meals and corrections a lot of insulin to take', {'entities': [(75, 82, 'DRUGS', 'novolog'), (118, 125, 'DRUGS', 'insulin'), (16, 22, 'DRUGS', 'lantus'), (54, 60, 'DRUGS', 'lantus'), (136, 142, 'DRUGS', 'lantus'), (30, 37, 'FINDINGS', 'morning'), (64, 69, 'FINDINGS', 'night')]})